In [1]:
from imports import *
from PreProcessor import *
import os
import numpy as np

In [2]:

CodeTokens = set()
UCTokens = set()
UC_documents = list()
code_documents = list()
TotalTokens=set()
entropy_uc = []
entropy_code = []
variance_uc = {}
variance_code = {}
SCQ_uc={}
SCQ_code={}
term_co_occurrences_uc = {}
PMI_uc = {}
term_co_occurrences_code= {}
PMI_code = {}


In [3]:
# Q, the set of query terms; q, a term in the query; D, the set of documents in the collection;
# Dt, the set of documents containing term t
# d, a document in the document collection D;
# tf(t, D), the frequency of term t in all docs;
# tf(t, d), the frequency of term t in d;
# tf(t, Q), the frequency of term t in the query;
# sim(di, dj), the cosine similarity between the vector-space representations of di and dj
# idf(t) = log( |D|/|Dt|).
# ictf(t) = log( |D|/tf(t,D) ).


In [4]:
for filename in os.listdir("./UC"):
    filepath = os.path.join("./UC", filename)
    tokens = PreProcessor().UCPreProcessor(filepath)
    UC_documents.append(tokens)
    UCTokens.update(tokens.split())


for filename in os.listdir("./CC"):
    filepath = os.path.join("./CC", filename)
    tokens = PreProcessor().CodePreProcessor(filepath)
    code_documents.append(tokens)
    CodeTokens.update(tokens.split())

TotalTokens = CodeTokens.union(UCTokens)


In [5]:
vectorizer_uc = TfidfVectorizer(vocabulary=TotalTokens)
tf_matrix_uc=vectorizer_uc.fit_transform(UC_documents)
idf_uc = vectorizer_uc.idf_
feature_names = vectorizer_uc.get_feature_names_out() 
idf_uc_dict = {feature_names[i]: idf_uc[i] for i in range(len(feature_names))}  #this is the idf dictionary for the whole vocab that i acsess later doing whatever

tf_uc_array = tf_matrix_uc.toarray().sum(axis=0)
tf_uc_dict = {feature_names[i]: tf_uc_array[i] for i in range(len(feature_names))}


df_uc_array = np.sum(tf_matrix_uc > 0, axis=0).A1
df_uc_dict = {feature_names[i]: df_uc_array[i] for i in range(len(feature_names))}

vectorizer_code = TfidfVectorizer(vocabulary=TotalTokens)
tf_matrix_code=vectorizer_code.fit_transform(code_documents)
idf_code = vectorizer_code.idf_
feature_names = vectorizer_code.get_feature_names_out() 
idf_code_dict = {feature_names[i]: idf_code[i] for i in range(len(feature_names))}


tf_code_array = tf_matrix_code.toarray().sum(axis=0)
tf_code_dict = {feature_names[i]: tf_code_array[i] for i in range(len(feature_names))}


df_code_array = np.sum(tf_matrix_code > 0, axis=0).A1
df_code_dict = {feature_names[i]: df_code_array[i] for i in range(len(feature_names))}




# IDF 


In [6]:
idf_uc_q=[]
for Querey in UC_documents:
    tokens = Querey.split()
    idf_q=[]
    for token in tokens:
        idf_q.append(idf_code_dict[token]) 
    idf_uc_q.append(idf_q)

idf_code_q=[]
for Querey in code_documents:
    tokens = Querey.split()
    idf_q=[]
    for token in tokens:
        idf_q.append(idf_uc_dict[token])
    idf_code_q.append(idf_q)



def AvgIDF(idf_values):
    avgidf=[]
    for query in idf_values:
        avgidf.append(sum(query)/len(idf_values))    # |G| is len(idf_values)
    return avgidf

def MaxIDF(idf_values):
    maxidf=[]
    for query in idf_values:
        maxidf.append(max(query))
    return maxidf


def DevIDF(idf_values):
    devidf=[]
    avgidf=AvgIDF(idf_values)
    for query_idf in idf_values:
        deviation = np.sqrt(sum([abs(idf - avg) for idf, avg in zip(query_idf, avgidf)]) / len(idf_values))
        devidf.append(deviation)
    return devidf
       


In [7]:
print(tf_code_dict)

{"'alreadi": 0.0, "'began": 0.0, "'remov": 0.0, "'s": 0.0, "'wa": 0.0, '..': 0.0, 'aar': 0.055891542948187864, 'abcdefghijklmnopqrstuvxywz': 0.03337123189713623, 'abstract': 0.3488642248456807, 'ac': 0.006252995005756665, 'accept': 0.10905115701654208, 'access': 0.11166764753941599, 'accord': 0.19306598288750254, 'account': 0.17471207664465568, 'act': 0.015117554487593499, 'action': 0.7437458658809549, 'activ': 1.1130755622679003, 'activateal': 0.0062248905056393345, 'active/deactiv': 0.0, 'activeagr': 0.0, 'actor': 0.021138647861694705, 'actual': 0.08112229896926941, 'ad': 0.17125924361024708, 'adapt': 0.14323271248659963, 'add': 2.0082942278830775, 'addit': 0.07642853329015156, 'address': 0.5046296109190818, 'adjust': 0.03901439110558455, 'administr': 0.011126620213691386, 'admiss': 0.004638330625677956, 'advanc': 0.6400038335512485, 'advertis': 2.0146583111752485, 'affili': 0.006906717285179304, 'afil': 0.11151915486740158, 'after': 0.021991555932838365, 'ag': 0.0018883946832802176,

# ICTF

In [8]:
#ictf(t) = log( |D|/tf(t,D) ).
ictf_uc_q=[]
for Querey in UC_documents:
    tokens = Querey.split()
    ictf_q=[]
    for token in tokens:
        if token in tf_code_dict.keys():
            ictf_q.append(np.log(len(code_documents)/(tf_code_dict[token]+1)))
        else :
            ictf_q.append(0) 
    ictf_uc_q.append(ictf_q)

ictf_code_q=[]
for Querey in code_documents:
    tokens = Querey.split()
    ictf_q=[]
    for token in tokens:
        if token in tf_uc_dict.keys():
           ictf_q.append(np.log(len(UC_documents)/(tf_uc_dict[token]+1)))
        else:
            ictf_q.append(0) 
    ictf_code_q.append(ictf_q)



def AvgICTF(ictf_values):
    avgictf=[]
    for query in ictf_values:
        avgictf.append(sum(query) / len(query))
    return avgictf

def MaxICTF(ictf_values):
    maxictf=[]
    for query in ictf_values:
        maxictf.append(max(query))
    return maxictf

def DevICTF(ictf_values):
    devictf=[]
    avgictf=AvgICTF(ictf_values)
    for query_ictf in ictf_values:
        deviation = np.sqrt(sum([abs(ictf - avg)for ictf ,avg in zip(query_ictf,avgictf)]) / len(ictf_values))
        devictf.append(deviation) 
    return devictf






# entropy

In [9]:
#HASSS THE SAME ISSUE IN QS OF SHIT ANF SH


# Q, the set of query terms; q, a term in the query; D, the set of documents in the collection;
# Dt, the set of documents containing term t
# d, a document in the document collection D;
# tf(t, D), the frequency of term t in all docs;
# tf(t, d), the frequency of term t in d;                
# tf(t, Q), the frequency of term t in the query;
# sim(di, dj), the cosine similarity between the vector-space representations of di and dj
# idf(t) = log( |D|/|Dt|).
# ictf(t) = log( |D|/tf(t,D) ).
#entropy(t) = sumd∈Dt(tf(t,d)/tf(t,D) · log|D|tf(t,d)/tf(t,D)  
entropy_uc = [] 

for query in UC_documents:
    tokens = query.split()
    query_entropy = []  
    for term in tokens:
        term_entropy =0
        for doc in code_documents:
            tf_term_doc = doc.count(term)+1
            tf_term_collection = df_code_dict[term] +1
            term_entropy+=((tf_term_doc / tf_term_collection) * np.log((tf_term_doc / tf_term_collection) + 1))
        query_entropy.append(term_entropy)
    entropy_uc.append(query_entropy)  


entropy_code = []
for query in code_documents:
    tokens = query.split()
    query_entropy = []  
    for term in tokens:
        term_entropy =0
        for doc in UC_documents:
            tf_term_doc = doc.count(term)+1
            tf_term_collection = df_uc_dict[term] +1
            term_entropy+=((tf_term_doc / tf_term_collection) * np.log((tf_term_doc / tf_term_collection) + 1))
        query_entropy.append(term_entropy)
    entropy_code.append(query_entropy)




def AvgEntropy(entropy_values):
    avgentropy=[]
    for query in entropy_values:
        avgentropy.append(sum(query) / len(entropy_values))
    return avgentropy
    
def MaxEntropy(entropy_values):
    maxentropy=[]
    for query in entropy_values:
        maxentropy.append(max(query))
    return maxentropy
def MedEntropy(entropy_values):
    medentropy=[]
    for query in entropy_values:
        medentropy.append(np.median(query))
    return medentropy



def DevEntropy(entropy_values):
    deventropy=[]
    avgentropy = AvgEntropy(entropy_values)
    for query_entropy in entropy_values:
        deviation = np.sqrt(sum([abs(entropy - avg) for entropy, avg in zip(query_entropy,avgentropy)]) / len(entropy_values))
        deventropy.append(deviation)
    return deventropy

# VAR

In [10]:
variance_uc = [] 

for query in UC_documents:
    tokens = query.split()
    query_variances = []  
    for term in tokens:
        term_weights = []
        for doc in code_documents:
            tf_term_doc = doc.count(term)
            weight_term_doc = (1 / len(doc)) * np.log(1 + tf_term_doc) * idf_uc_dict[term]  #
            term_weights.append(weight_term_doc)

        avg_weight_term = np.mean(term_weights)
        variance_term = np.mean([(weight - avg_weight_term) ** 2 for weight in term_weights])
        query_variances.append(variance_term) 
    variance_uc.append(query_variances)  

variance_code = []

for query in code_documents:
    tokens = query.split()
    query_variances = []  
    for term in tokens:
        term_weights = []
        for doc in UC_documents:
            tf_term_doc = doc.count(term)
            weight_term_doc = (1 / len(doc)) * np.log(1 + tf_term_doc) * idf_code_dict[term]  #
            term_weights.append(weight_term_doc)

        avg_weight_term = np.mean(term_weights)
        variance_term = np.mean([(weight - avg_weight_term) ** 2 for weight in term_weights])
        query_variances.append(variance_term) 
    variance_code.append(query_variances)



def AvgVariance(variance_values):
    avgvariance=[]
    for query in variance_values:
        avgvariance.append(sum(query) / len(variance_values))
    return avgvariance

def MaxVariance(variance_values):
    maxvariance=[]
    for query in variance_values:
        maxvariance.append(max(query))
    return maxvariance

def SumVariance(variance_values):
    sumvariance=[]
    for query in variance_values:
        sumvariance.append(sum(query))
    return sumvariance


# SCQ

In [11]:


SCQ_uc = []  
for query in UC_documents:
    tokens = query.split()
    query_scq = []  
    for term in tokens:
        if term in tf_code_dict.keys() and term in idf_code_dict.keys():
          query_scq.append((1 + np.log((tf_code_dict[term])+1) * idf_code_dict[term]))
        else:
            query_scq.append(0)  
    SCQ_uc.append(query_scq)


SCQ_code = []

for query in code_documents:
    tokens = query.split()
    query_scq = []  
    for term in tokens:
        if term in tf_uc_dict.keys() and term in idf_uc_dict.keys():
            query_scq.append((1 + np.log((tf_uc_dict[term])+1) * idf_uc_dict[term])+1)
        else:
            query_scq.append(0)
    SCQ_code.append(query_scq)


def AvgSCQ(SCQ_values):
    avgscq=[]
    for query in SCQ_values:
        avgscq.append(sum(query) / len(SCQ_values))
    return avgscq

def MaxSCQ(SCQ_values):
    maxscq=[]
    for query in SCQ_values:
        maxscq.append(max(query))
    return maxscq

def SumSCQ(SCQ_values):
    sumscq=[]
    for query in SCQ_values:
        sumscq.append(sum(query))
    return sumscq


# PMI

In [12]:
from collections import defaultdict
PMI_code = [] 

for doc in code_documents:
    unique_terms = set(doc)
    
    term_co_occurrences_code = defaultdict(int)
    for term1 in unique_terms:
        for term2 in unique_terms:
            if term1 != term2:
                term_co_occurrences_code[(term1, term2)] += 1

    document_PMI = {} 
    for term_pair, co_occurrence_count in term_co_occurrences_code.items():
        term1, term2 = term_pair
        pt1_t2_D = co_occurrence_count / len(doc)
        pt1_D = sum(1 for term in doc if term == term1) / len(doc)
        pt2_D = sum(1 for term in doc if term == term2) / len(doc)
        pt_D = pt1_D * pt2_D

        if pt_D != 0:
            document_PMI[term_pair] = np.log(pt1_t2_D / pt_D)
    PMI_code.append(document_PMI)

PMI_uc = []

for doc in UC_documents:
    unique_terms = set(doc)
    
    term_co_occurrences_uc = defaultdict(int)
    for term1 in unique_terms:
        for term2 in unique_terms:
            if term1 != term2:
                term_co_occurrences_uc[(term1, term2)] += 1

    document_PMI = {} 
    for term_pair, co_occurrence_count in term_co_occurrences_uc.items():
        term1, term2 = term_pair
        pt1_t2_D = co_occurrence_count / len(doc)
        pt1_D = sum(1 for term in doc if term == term1) / len(doc)
        pt2_D = sum(1 for term in doc if term == term2) / len(doc)
        pt_D = pt1_D * pt2_D

        if pt_D != 0:
            document_PMI[term_pair] = np.log(pt1_t2_D / pt_D)
    PMI_uc.append(document_PMI)


def AvgPMI(PMI_values):
    avgpmi=[]
    for query in PMI_values:
        num_terms = len(query)
        if num_terms > 1:
            avgpmi.append(2 * sum(query.values()) * math.exp(math.lgamma(num_terms - 1) - math.lgamma(num_terms)))
        else:
            avgpmi.append(0) 
    return avgpmi

def MaxPMI(PMI_values):
    maxpmi=[]
    for query in PMI_values:
        if query:
            maxpmi.append(max(query.values()))
        else:
            maxpmi.append(0)
    return maxpmi

# QS

In [13]:
#The percentage of documents in the collection containing at least one of the query terms
def QS(documents, otherdocuments):
    QSdocs = []
    total_documents = len(otherdocuments)
    
    for query in documents:
        term_occurrence = set() 
        tokens = query.split() 
        for term in tokens:
            for doc in otherdocuments:
                if term in doc:
                    term_occurrence.add(doc)
        
        percentage = (len(term_occurrence) / total_documents) * 100
        QSdocs.append(percentage)
    return QSdocs

# CS

In [14]:

# we try construct the first for loops out into a vector of each querey and it's corresponding vector of term documents b3daha n move forword
#where |Dq| is the size of that corresponding vector 


def compute_term_info(documents):
    term_info = {}
    for query in documents:
        for term in query:
            if term not in term_info:
                term_docs = [doc for doc in documents if term in doc]
                term_info[term] = (term_docs, len(term_docs))
    return term_info

def CS(documents, tf_matrix, term_info):
    CSdocs = []
    for query in documents:
        term_similarities = []
        for term in query:
            term_docs, num_term_docs = term_info[term]
            if num_term_docs > 1:
                similarity_sum = 0
                pairs_count = 0
                for i, doc1 in enumerate(term_docs):
                    for j, doc2 in enumerate(term_docs):
                        if i != j:
                            doc1_index = documents.index(doc1)
                            doc2_index = documents.index(doc2)
                            similarity = cosine_similarity(tf_matrix[doc1_index], tf_matrix[doc2_index])
                            similarity_sum += similarity
                            pairs_count += 1
                
                if pairs_count > 0:
                    term_similarities.append(similarity_sum / pairs_count)
        
        if term_similarities:
            CSdocs.append(np.mean(term_similarities))
        else:
            CSdocs.append(0)
    
    return CSdocs


In [15]:

# IDF
avg_idf = AvgIDF(idf_uc_q)
print("Average IDF for UC documents:", avg_idf)

max_idf = MaxIDF(idf_uc_q)
print("Max IDF for UC documents:", max_idf)

dev_idf = DevIDF(idf_uc_q)
print("Deviation of IDF for UC documents:", dev_idf)

# ICTF
avg_ictf = AvgICTF(ictf_uc_q)
print("Average ICTF for UC documents:", avg_ictf)

max_ictf = MaxICTF(ictf_uc_q)
print("Max ICTF for UC documents:", max_ictf)

dev_ictf = DevICTF(ictf_uc_q)
print("Deviation of ICTF for UC documents:", dev_ictf)

# Entropy
avg_entropy = AvgEntropy(entropy_uc)
print("Average Entropy for UC documents:", avg_entropy)

max_entropy = MaxEntropy(entropy_uc)
print("Max Entropy for UC documents:", max_entropy)

med_entropy = MedEntropy(entropy_uc)
print("Median Entropy for UC documents:", med_entropy)

dev_entropy = DevEntropy(entropy_uc)
print("Deviation of Entropy for UC documents:", dev_entropy)

# Variance
avg_variance = AvgVariance(variance_uc)
print("Average Variance for UC documents:", avg_variance)

max_variance = MaxVariance(variance_uc)
print("Max Variance for UC documents:", max_variance)

sum_variance = SumVariance(variance_uc)
print("Sum of Variance for UC documents:", sum_variance)

# SCQ
avg_scq = AvgSCQ(SCQ_uc)
print("Average SCQ for UC documents:", avg_scq)

max_scq = MaxSCQ(SCQ_uc)
print("Max SCQ for UC documents:", max_scq)

sum_scq = SumSCQ(SCQ_uc)
print("Sum of SCQ for UC documents:", sum_scq)

# PMI
avg_pmi = AvgPMI(PMI_uc)
print("Average PMI for UC documents:", avg_pmi)

max_pmi = MaxPMI(PMI_uc)
print("Max PMI for UC documents:", max_pmi)

# QS
qs_uc = QS(UC_documents,code_documents)
print("QS for UC documents:", qs_uc)

# # CS
term_info_cs_uc = compute_term_info(UC_documents)
cs_uc = CS(UC_documents, tf_matrix_uc, term_info_cs_uc)
print("CS for UC documents:", cs_uc)

# IDF
avg_idf = AvgIDF(idf_code_q)
print("Average IDF for Code documents:", avg_idf)

max_idf = MaxIDF(idf_code_q)
print("Max IDF for Code documents:", max_idf)

dev_idf = DevIDF(idf_code_q)
print("Deviation of IDF for Code documents:", dev_idf)

# ICTF
avg_ictf = AvgICTF(ictf_code_q)
print("Average ICTF for Code documents:", avg_ictf)

max_ictf = MaxICTF(ictf_code_q)
print("Max ICTF for Code documents:", max_ictf)

dev_ictf = DevICTF(ictf_code_q)
print("Deviation of ICTF for Code documents:", dev_ictf)

# Entropy
avg_entropy = AvgEntropy(entropy_code)
print("Average Entropy for Code documents:", avg_entropy)

max_entropy = MaxEntropy(entropy_code)
print("Max Entropy for Code documents:", max_entropy)

med_entropy = MedEntropy(entropy_code)
print("Median Entropy for Code documents:", med_entropy)

dev_entropy = DevEntropy(entropy_code)
print("Deviation of Entropy for Code documents:", dev_entropy)

# Variance
avg_variance = AvgVariance(variance_code)
print("Average Variance for Code documents:", avg_variance)

max_variance = MaxVariance(variance_code)
print("Max Variance for Code documents:", max_variance)

sum_variance = SumVariance(variance_code)
print("Sum of Variance for Code documents:", sum_variance)

# SCQ
avg_scq = AvgSCQ(SCQ_code)
print("Average SCQ for Code documents:", avg_scq)

max_scq = MaxSCQ(SCQ_code)
print("Max SCQ for Code documents:", max_scq)

sum_scq = SumSCQ(SCQ_code)
print("Sum of SCQ for Code documents:", sum_scq)

# PMI
avg_pmi = AvgPMI(PMI_code)
print("Average PMI for Code documents:", avg_pmi)

max_pmi = MaxPMI(PMI_code)
print("Max PMI for Code documents:", max_pmi)

# QS
qs_code = QS(code_documents,UC_documents)
print("QS for Code documents:", qs_code)

# # CS
term_info_cs_code = compute_term_info(code_documents)
cs_code = CS(code_documents, tf_matrix_code, term_info_cs_code)
print("CS for Code documents:", cs_code)




Average IDF for UC documents: [3.135923404374892, 2.2378565767484444, 2.4901381669491762, 2.109964426731631, 2.386672736422987, 2.275791491813192, 3.508046618930981, 2.2481665766142207, 4.56266759133216, 3.0024311408082442, 3.349398518272376, 3.800379856333389, 4.986315702686, 3.396799057045237, 2.5945922042538982, 3.9482854287795766, 2.165247611925174, 3.144316739010335, 4.360434449666477, 2.207150918221824, 2.444381699216136, 2.243580209134455, 4.228053065287774, 3.2674841468288602, 2.585446216214325, 1.8971285499683552, 3.529658210868321, 2.4039473298081657, 3.1292474395255128, 2.1167377292724243, 2.313123850335303, 4.640506974226651, 4.468057966810239, 2.41457313902866, 3.2310958369464706, 4.5629588756720425, 3.214121016664666, 3.718704462331911, 3.9620968609128115, 2.1095853247020697, 3.165442948360561, 3.4177468035598717, 3.107873881261588, 1.9695674429492116, 2.047928467349927, 2.094149169594623, 2.4453018967811393, 2.6758711710072713, 5.077639275643823, 3.0297742263378815, 2.24